# Projects in ML HW 6

## Task 1

We discussed how we can formulate RL problems as an MDP. Describe any
real-world application that can be formulated as an MDP. Describe the state space, action
space, transition model, and rewards for that problem. You do not need to be precise in the
description of the transition model and reward (no formula is needed). Qualitative description
is enough.

One real-world application of MDP I can think of is any type of diasastor control. I think nuclear powerplants are a well known example of such an application. 

The **state space** of a nuclear powerplant can be state of its sensors.

The **action space** of a nuclear poewrplant can be a possible set of actions that can be automated. (i.e. alert operator, stop current action, cooldown mode, etc)

The **transition model** can be composed of the sensor outputs *after* the action is changed. For example, if the coolant is too hot `{...state, coolanttemp: 99}`, the state transition after the action of cooling it down could be into `{...state, coolanttemp: 50}`. Each sensor or group of sensors should have some action linked to it, in the example it is the case of the 'coolant temp sensor' and 'cooling system'.

The **reward** in this case would be minimizing risk, as nuclear powerplants generally produce plenty power, but a single mistake could bring devastating outcome

## Task 2 

RL is used in various sectors - Healthcare, recommender systems and trading
are a few of those. Pick one of the three areas. Explain one of the problems in any of these
domains that can be more effectively solved by reinforcement learning. Find an open-source
project (if any) that has addressed this problem. Explain this project in detail.

Reinforcement learning is well suited to healtcare, as the physician needs to continue monitoring the patient's state and choose the best actions to improve their state. One problem that exists in this domain is that there is often not enough data to sufficiently train a model, due to the sensitive nature of the domain.

I found this open source project from Microsoft called "Med-Deadend" https://github.com/microsoft/med-deadend. 

This project addresses a problem in ML with healthcare -- lack of training data -- and re-scopes the problem so that a good model can be learned with smaller amounts of data. This project focuses on reinforcement learning on data from critically ill patients, and learns to predict a "dead-end", which means the patient will go in critical condition. In essence, the purpose of this model is to identify which procedures must be taken to **prevent** such dead ends. 

The problem is modeled such that a positive outcome gets a reward of 1, and a negative outcome gets a reward of -1. The state space is compoesd of various features of a patient, such as their blodd pressure or hear rate. They also make use of a specific variety of reinforcement learning, called offline reinforcement learning where the learning happens with existing data. Usually, this approach can still take a large amount of data to train sufficiently, but by scoping the problem to a specific domain, the authors show that their model does indeed work. 

Furthur details can also be found in https://www.microsoft.com/en-us/research/blog/using-reinforcement-learning-to-identify-high-risk-states-and-treatments-in-healthcare/


## Task 3

Implement the game of tic-tac-toe (write a class that implements an agent playing Tic Tac Toe and learning its Q function) using the Q-learning technique (see the resources/links provided in class for more details). Clearly describe your evaluation metric and demonstrate a few runs. You might need to use some online resources to proceed on this. Do not forget to cite those.

In [ ]:
#Example- Frozen lake: Import the Gym Library
import gym
import matplotlib.pyplot as plt
import time